In [8]:
import pandas as pd
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA, BollingerBands
from indicators import time_to_epoch, epoch_to_time
import datetime, time
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [9]:
import requests

def get_market_rates(symbol:str, timeframe:str, from_candle:int=0, previous_candle_count:int=100, eohlcv:bool=True, base_url:str="http://10.168.85.228:7767/data") -> np.ndarray:
    if timeframe not in ["1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"]:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')

    if timeframe == "1m":
        _tf = 1
    elif timeframe == "5m":
        _tf = 5
    elif timeframe == "15m":
        _tf = 15
    elif timeframe == "30m":
        _tf = 30
    elif timeframe == "1h":
        _tf = 16385
    elif timeframe == "1d":
        _tf = 16408
    elif timeframe == "4h":
        _tf = 16388
    elif timeframe == "1w":
        _tf = 32769
    elif timeframe == "1mo":
        _tf = 49153
    else:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')
    payload = {
        "symbol": symbol,
        "timeframe": _tf,
        "from_candle": from_candle,
        "previous": previous_candle_count
    }

    response = requests.post(base_url, json=payload)

    if response.status_code == 200:
        _data = response.json()['rates']
        # print("Bars received:", len(_data))
        if eohlcv:
            return np.array(_data)[:, :6]
        else:
            return np.array(_data)
    else:
        print("Error:", response.text)
        raise RuntimeError(response.text)

def get_market_range_rates(symbol:str, timeframe:str, from_candle:str="31-12-2025 00:00", previous_candle_count:int=100, eohlcv:bool=True, base_url:str="http://10.168.85.228:7767/data-range") -> np.ndarray:
    if timeframe not in ["1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"]:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')

    if timeframe == "1m":
        _tf = 1
    elif timeframe == "5m":
        _tf = 5
    elif timeframe == "15m":
        _tf = 15
    elif timeframe == "30m":
        _tf = 30
    elif timeframe == "1h":
        _tf = 16385
    elif timeframe == "1d":
        _tf = 16408
    elif timeframe == "4h":
        _tf = 16388
    elif timeframe == "1w":
        _tf = 32769
    elif timeframe == "1mo":
        _tf = 49153
    else:
        raise Exception('''Timeframe should be from -> "1m", "5m", "15m", "30m", "1h", "4h", "1d", "1w", "1mo"''')
    payload = {
        "symbol": symbol,
        "timeframe": _tf,
        "from-time": from_candle,
        "previous": previous_candle_count
    }

    response = requests.post(base_url, json=payload)

    if response.status_code == 200:
        _data = response.json()['rates']
        # print("Bars received:", len(_data))
        if eohlcv:
            return np.array(_data)[:, :6]
        else:
            return np.array(_data)
    else:
        print("Error:", response.text)
        raise RuntimeError(response.text)

In [10]:
data_1h = get_market_rates("XAUUSD", timeframe="1h", previous_candle_count=100)
data_1h.shape

(100, 6)

In [5]:
data_4h = get_market_rates("XAUUSD", timeframe="4h", previous_candle_count=100)
data_4h.shape

(100, 6)

In [6]:
data_1d = get_market_rates("XAUUSD", timeframe="1d", previous_candle_count=100)
data_1d.shape

(100, 6)

In [7]:
data_1h = get_market_rates("XAUUSD", timeframe="1h", previous_candle_count=100)
data_4h = get_market_rates("XAUUSD", timeframe="4h", previous_candle_count=100)
data_1d = get_market_rates("XAUUSD", timeframe="1d", previous_candle_count=100)

print(f"Current time is {epoch_to_time(data_1h[-1, 0], timezone="UTC")} and current price is {data_1h[-1, 4]}")


print("Following are the 100 previous candles on timeframe of 1 hour:")
for i in data_1h:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print("\nFollowing are the 100 previous candles on timeframe of 4 hour:")
for i in data_4h:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print("\nFollowing are the 100 previous candles on timeframe of 1 day:")
for i in data_1d:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


Current time is 2026-01-06 18:00:00 and current price is 4481.18
Following are the 100 previous candles on timeframe of 1 hour:
2025-12-30 11:00:00 4365.03 4376.51 4364.12 4372.24 23073.0
2025-12-30 12:00:00 4372.17 4391.73 4367.76 4384.86 23814.0
2025-12-30 13:00:00 4384.86 4390.26 4381.26 4384.32 22345.0
2025-12-30 14:00:00 4384.3 4402.81 4383.5 4401.23 24203.0
2025-12-30 15:00:00 4401.3 4404.25 4376.36 4388.46 39864.0
2025-12-30 16:00:00 4388.46 4392.17 4355.91 4366.37 42575.0
2025-12-30 17:00:00 4366.33 4377.53 4359.55 4365.08 39038.0
2025-12-30 18:00:00 4365.25 4379.91 4354.9 4377.81 31622.0
2025-12-30 19:00:00 4377.81 4381.82 4364.36 4370.21 29379.0
2025-12-30 20:00:00 4370.23 4378.68 4362.68 4364.58 26443.0
2025-12-30 21:00:00 4364.58 4368.01 4337.37 4348.61 24526.0
2025-12-30 22:00:00 4348.61 4355.53 4337.78 4341.74 27674.0
2025-12-30 23:00:00 4341.71 4350.28 4334.81 4339.12 8780.0
2025-12-31 01:00:00 4340.58 4346.61 4328.71 4339.01 16688.0
2025-12-31 02:00:00 4339.04 4346.68 4

In [56]:
data_5m = get_market_rates("XAUUSD", timeframe="5m", previous_candle_count=100)
data_15m = get_market_rates("XAUUSD", timeframe="15m", previous_candle_count=100)
data_1h = get_market_rates("XAUUSD", timeframe="1h", previous_candle_count=100)
data_4h = get_market_rates("XAUUSD", timeframe="4h", previous_candle_count=100)
data_1d = get_market_rates("XAUUSD", timeframe="1d", previous_candle_count=100)

print(f"Current time is {epoch_to_time(time.time(), timezone="Europe/Nicosia")} and current price is {data_1h[-1, 4]}")

# print("\nFollowing are the 100 previous candles on timeframe of 5 minutes:")
# for i in data_5m:
#     print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])

print("\nFollowing are the 100 previous candles on timeframe of 15 minutes:")
for i in data_15m:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print("\nFollowing are the 100 previous candles on timeframe of 1 hour:")
for i in data_1h:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print("\nFollowing are the 100 previous candles on timeframe of 4 hour:")
for i in data_4h:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print("\nFollowing are the 100 previous candles on timeframe of 1 day:")
for i in data_1d:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


Current time is 2026-01-05 10:50:19 and current price is 4432.27

Following are the 100 previous candles on timeframe of 15 minutes:
2026-01-02 09:00:00 4374.77 4378.0 4372.37 4377.31 4250.0
2026-01-02 09:15:00 4377.33 4377.48 4371.99 4373.87 4242.0
2026-01-02 09:30:00 4373.87 4380.71 4372.24 4375.8 4568.0
2026-01-02 09:45:00 4375.8 4383.4 4375.34 4382.82 5315.0
2026-01-02 10:00:00 4382.82 4386.64 4380.82 4385.13 4089.0
2026-01-02 10:15:00 4385.14 4385.77 4381.69 4385.35 3751.0
2026-01-02 10:30:00 4385.35 4390.6 4384.19 4390.6 4228.0
2026-01-02 10:45:00 4390.6 4397.65 4386.86 4387.0 5799.0
2026-01-02 11:00:00 4387.0 4391.14 4386.18 4390.08 4498.0
2026-01-02 11:15:00 4390.06 4391.61 4383.32 4388.71 4662.0
2026-01-02 11:30:00 4388.71 4389.85 4386.85 4388.75 3698.0
2026-01-02 11:45:00 4388.81 4389.56 4385.08 4385.55 4025.0
2026-01-02 12:00:00 4385.54 4388.47 4384.59 4388.12 2974.0
2026-01-02 12:15:00 4388.12 4390.04 4385.63 4386.4 3194.0
2026-01-02 12:30:00 4386.4 4396.58 4384.89 4393.71 

In [15]:
last_count = 100
data_1m = get_market_rates("XAUUSD", timeframe="1m", previous_candle_count=last_count)
data_5m = get_market_rates("XAUUSD", timeframe="5m", previous_candle_count=last_count)
data_15m = get_market_rates("XAUUSD", timeframe="15m", previous_candle_count=last_count)
data_30m = get_market_rates("XAUUSD", timeframe="30m", previous_candle_count=last_count)
data_1h = get_market_rates("XAUUSD", timeframe="1h", previous_candle_count=last_count)
data_4h = get_market_rates("XAUUSD", timeframe="4h", previous_candle_count=last_count)
data_1d = get_market_rates("XAUUSD", timeframe="1d", previous_candle_count=last_count)

print(f"Current time is {epoch_to_time(time.time(), timezone="Europe/Nicosia")} and current price is {data_1h[-1, 4]}")

print(f"\nFollowing are the {last_count} previous candles on timeframe of 1 minutes:")
for i in data_1m:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print(f"\nFollowing are the {last_count} previous candles on timeframe of 5 minutes:")
for i in data_5m:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])

print(f"\nFollowing are the {last_count} previous candles on timeframe of 15 minutes:")
for i in data_15m:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print(f"\nFollowing are the {last_count} previous candles on timeframe of 30 minutes:")
for i in data_30m:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print(f"\nFollowing are the {last_count} previous candles on timeframe of 1 hour:")
for i in data_1h:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print(f"\nFollowing are the {last_count} previous candles on timeframe of 4 hour:")
for i in data_4h:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


print(f"\nFollowing are the {last_count} previous candles on timeframe of 1 day:")
for i in data_1d:
    print(epoch_to_time(i[0], timezone="UTC"), i[1], i[2], i[3], i[4], i[5])


Current time is 2026-01-06 18:50:02 and current price is 4485.57

Following are the 100 previous candles on timeframe of 1 minutes:
2026-01-06 17:11:00 4483.29 4484.46 4480.02 4481.19 817.0
2026-01-06 17:12:00 4481.19 4482.69 4479.13 4480.17 763.0
2026-01-06 17:13:00 4480.1 4483.17 4479.41 4483.12 642.0
2026-01-06 17:14:00 4483.12 4484.57 4480.19 4480.25 472.0
2026-01-06 17:15:00 4480.25 4482.68 4480.2 4482.68 592.0
2026-01-06 17:16:00 4482.65 4483.36 4479.94 4480.05 535.0
2026-01-06 17:17:00 4480.05 4482.07 4477.34 4477.89 649.0
2026-01-06 17:18:00 4477.89 4479.75 4477.33 4479.0 619.0
2026-01-06 17:19:00 4479.0 4479.01 4474.51 4474.55 671.0
2026-01-06 17:20:00 4474.57 4476.05 4473.87 4476.05 709.0
2026-01-06 17:21:00 4476.06 4477.95 4475.43 4477.85 712.0
2026-01-06 17:22:00 4477.85 4478.84 4476.02 4477.56 630.0
2026-01-06 17:23:00 4477.56 4479.01 4477.17 4478.34 578.0
2026-01-06 17:24:00 4478.34 4478.9 4475.56 4476.65 727.0
2026-01-06 17:25:00 4476.65 4478.42 4476.21 4477.94 638.0
202

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

PROMPT = "Hello DeepSeek, write a haiku about AI."

# Setup Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get("https://chat.deepseek.com")

wait = WebDriverWait(driver, 30)

# Wait for the input textarea (selector may change)
input_box = wait.until(
    EC.presence_of_element_located((By.TAG_NAME, "textarea"))
)

# Type prompt and press Enter
input_box.send_keys(PROMPT)
input_box.send_keys(Keys.ENTER)

# Optional: keep browser open
time.sleep(10)


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

PROMPT = "Write a short imaginary story about artificial intelligence."

# Setup Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 40)

# Open DeepSeek Chat
driver.get("https://chat.deepseek.com")

# Wait for input box
input_box = wait.until(
    EC.presence_of_element_located((By.TAG_NAME, "textarea"))
)

# Send prompt
input_box.send_keys(PROMPT)
input_box.send_keys(Keys.ENTER)

# ---- WAIT FOR RESPONSE ----
# This waits until at least one assistant message appears
wait.until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, "div[class*='message'], div[class*='assistant']")
    )
)

# Give extra time for streaming to finish
time.sleep(5)

# ---- EXTRACT RESPONSE ----
# Grab all visible messages
messages = driver.find_elements(
    By.CSS_SELECTOR,
    "div[class*='message'], div[class*='assistant']"
)

# Print last assistant response
if messages:
    last_message = messages[-1].text
    print("\n===== DeepSeek Response =====\n")
    print(last_message)
else:
    print("No response found.")

# Keep browser open briefly
time.sleep(5)
driver.quit()



ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))